In [98]:
# %%writefile test.py

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
# from matplotlib.lines import Line2D
import pickle  
import time
from sklearn.cluster import KMeans

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
from bs4 import BeautifulSoup
import requests
import re


In [6]:
# Spotify credentials:

input_file = open("/Users/renev/OneDrive/Desktop/input.txt","r")
string = input_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]
auth_manager = SpotifyClientCredentials(client_id = secrets_dict['client_id'], 
                                        client_secret = secrets_dict['client_secret'])
sp = spotipy.Spotify(auth_manager=auth_manager)

In [7]:
# Load the model
filename = 'song_model.sav'
kmeans = pickle.load(open(filename, 'rb'))

In [8]:
#Load the scaler   
scaler = pickle.load(open('scaler.pkl', 'rb'))


In [9]:
# Load the database of songs
database_songs = pd.read_csv('database_songs.csv')

In [10]:
database_songs

,Unnamed: 0,id,name,album,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,clusters,url
0,0,000RW47rhEkSqjgTrZx7YX,Lance Individual,Tudo Em Paz,Jorge & Mateus,0.676500,-0.068031,0.546974,-0.555159,0.096825,-0.277117,-0.728997,1.995026,-0.140359,-0.603093,1,https://open.spotify.com/track/000RW47rhEkSqjg...
1,1,000xQL6tZNLJzIrtIgxqSl,Still Got Time (feat. PARTYNEXTDOOR),Still Got Time (feat. PARTYNEXTDOOR),ZAYN,0.650606,-0.032167,0.400269,-0.444424,-0.550907,-0.277117,-0.657884,0.109382,-0.015593,-0.316678,1,https://open.spotify.com/track/000xQL6tZNLJzIr...
2,2,000xYdQfIZ4pDmBGzQalKU,"Eu, Você, O Mar e Ela",1977,Luan Santana,-0.896561,0.869567,0.187800,-0.648005,1.461431,-0.274067,1.976087,0.656133,1.561659,-0.333030,3,https://open.spotify.com/track/000xYdQfIZ4pDmB...
3,3,0010ypQ2nPZdksx1kEjt66,As Nieko Nezinau,As Nieko Nezinau,Kamaniu Silelis,0.359299,-2.086685,-1.810124,-0.645450,2.207415,-0.116441,-0.797321,-1.642803,-0.086238,0.134967,6,https://open.spotify.com/track/0010ypQ2nPZdksx...
4,4,00115zIia5fhpbtl4c58RZ,Pokoj,Pokoj,Lenka Piešová,-1.136081,-1.456496,0.010147,-0.694003,-0.507240,-0.276759,-0.603504,-1.561006,-0.012407,-0.334448,4,https://open.spotify.com/track/00115zIia5fhpbt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207561,207561,7zzSBaZl2B3gvnln8h5xef,Три белых коня,Три белых коня,Ivanushki International,0.171567,1.771301,1.009110,-0.060261,-0.840931,-0.277117,1.097632,1.650616,1.002033,-0.321875,1,https://open.spotify.com/track/7zzSBaZl2B3gvnl...
207562,207562,7zzU5vd1GrHpNjwq3HpIe3,Le petit Nicolas,R.I.P.R.O 4,Lacrim,0.268670,0.562158,-0.352598,3.253255,-0.691007,-0.277117,-0.895625,-0.644014,0.711333,-0.088138,2,https://open.spotify.com/track/7zzU5vd1GrHpNjw...
207563,207563,7zzbnhCpuuHOCrhBGvgwpp,Wouldn’t Change A Thing,Need Of You,Lawrence Hancock,-0.961296,-0.590627,0.029787,-0.511717,-1.020732,-0.276977,-0.436179,-1.979033,-1.518839,-0.358880,4,https://open.spotify.com/track/7zzbnhCpuuHOCrh...
207564,207564,7zzhRAobAnNjarw2FAWt65,ארבעה אחים - פסח,שירי החגים הגדולים בכל הזמנים,Various Artists,0.022676,-1.482114,-0.966199,-0.673560,2.087330,-0.277117,-0.533785,1.607565,0.144250,-1.170296,6,https://open.spotify.com/track/7zzhRAobAnNjarw...


In [11]:
database_songs['clusters'].value_counts()

1    53012
4    44447
6    31312
2    28466
7    28345
3    11079
8     7693
0     3152
5       60
Name: clusters, dtype: int64

In [99]:
# User input → based on Spotify song url to avoid mis-typing, etc.

user_input = input("Please enter a Spotify track url: ")


Please enter a Spotify track url: https://open.spotify.com/track/0eTqauier0C0bigAwpmUWT


In [100]:
track_id = user_input[31:53]

In [101]:
track_id

'0eTqauier0C0bigAwpmUWT'

Collect the audio features from the Spotify API. 

In [102]:
#     track_info = sp.track(id)
#     track_features = sp.audio_features(id)

track_info = sp.track(track_id)
track_features = sp.audio_features(track_id)
    
#     Track info
name = track_info['name']
album= track_info['album']['name']
artist= track_info['album']['artists'][0]['name']
#     release_date= track_info['album']['release_date']
#     length= track_info['duration_ms']
#     popularity= track_info['popularity']
    
#     Track features

try: 
    danceability = track_features[0]['danceability']
    energy=track_features[0]['energy']
#     key=track_features[0]['key']
    loudness= track_features[0]['loudness']
#     mode=track_features[0]['mode']
    speechiness=track_features[0]['speechiness']
    acousticness= track_features[0]['acousticness']
    instrumentalness=track_features[0]['instrumentalness']
    liveness=track_features[0]['liveness']
    valence= track_features[0]['valence']
    tempo=track_features[0]['tempo']
    id= track_features[0]['id']
    duration_ms= track_features[0]['duration_ms']
#     time_signature= track_features[0]['time_signature']
    track_data = [id, name, album, artist, danceability,energy,loudness,speechiness, acousticness, instrumentalness,
                  liveness, valence, tempo, duration_ms]
except:
    
    track_data = [id,name, album, artist, 'null','null','null','null', 'null', 'null',
                  'null', 'null','null', 'null']

In [103]:
track_data

['0eTqauier0C0bigAwpmUWT',
 'Does Your Mother Know',
 'Abba Live',
 'ABBA',
 0.449,
 0.982,
 -6.504,
 0.0757,
 0.0441,
 0.000256,
 0.987,
 0.607,
 144.99,
 249733]

In [104]:
track_data = pd.DataFrame(track_data).T.values.tolist() #Transpose dataframe to list

In [105]:
features = pd.DataFrame(track_data, columns = ['id','name', 'album', 'artist', 'danceability','energy','loudness','speechiness',
                                               'acousticness','instrumentalness','liveness','valence', 'tempo', 'duration_ms'
                                               ])

In [106]:
features

,id,name,album,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0eTqauier0C0bigAwpmUWT,Does Your Mother Know,Abba Live,ABBA,0.449,0.982,-6.504,0.0757,0.0441,0.000256,0.987,0.607,144.99,249733


Check against Billboard Hot100

In [107]:
user_input_name = features['name'][0]

# select everything before brackets

In [108]:
user_input_name=re.sub(" \(.*?\)","",user_input_name)

In [75]:
user_input_name

'Does Your Mother Know'

In [76]:
url = "https://www.billboard.com/charts/hot-100"

In [77]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [78]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [79]:
hot_songs = []
# artist = []
num_iter = len(soup.select('span.chart-element__information span'))

for i in range(0,num_iter,7): # start at position 0, iterate through the len of the column, stop at 7
    hot_songs.append(soup.select('span.chart-element__information span')[i].get_text())
#     artist.append(soup.select('span.chart-element__information span')[i+1].get_text()) # artist is at position 0+1

In [80]:
hot_songs = pd.DataFrame(hot_songs)


In [109]:
# hot_songs[0]=hot_songs[0].str.extract(" \(.*?\)")

hot_songs[0] = hot_songs[0].str.replace(" \(.*?\)","", regex = True)

In [82]:
hot_songs_x = list(hot_songs[0])

After that, you want to send the Spotify audio features of the 
submitted song to the clustering model, which should return a cluster number.

In [83]:
# Change object to float

features['danceability'] = pd.to_numeric(features['danceability'],errors='coerce')
features['energy'] = pd.to_numeric(features['energy'],errors='coerce')
features['loudness'] = pd.to_numeric(features['loudness'],errors='coerce')
features['speechiness'] = pd.to_numeric(features['speechiness'],errors='coerce')
features['acousticness'] = pd.to_numeric(features['acousticness'],errors='coerce')
features['instrumentalness'] = pd.to_numeric(features['instrumentalness'],errors='coerce')
features['liveness'] = pd.to_numeric(features['liveness'],errors='coerce')
features['valence'] = pd.to_numeric(features['valence'],errors='coerce')
features['tempo'] = pd.to_numeric(features['tempo'],errors='coerce')
features['duration_ms'] = pd.to_numeric(features['duration_ms'],errors='coerce')

In [84]:
audio_features_less = features.drop(['id','name','album','artist'],axis=1) 

In [85]:
audio_features_less = pd.DataFrame(audio_features_less, index = [0])
audio_features_less

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.449,0.982,-6.504,0.0757,0.0441,0.000256,0.987,0.607,144.99,249733


In [86]:
X_prep_less = scaler.transform(audio_features_less)

In [87]:
X_prep_less

array([[-1.28497152,  1.78667107,  0.25892091, -0.3439116 , -0.86713171,
        -0.27566857,  5.62934547,  0.4667074 ,  0.82552632,  0.4132059 ]])

In [88]:
if user_input_name in hot_songs_x:
    print("May I suggest the following hot song:")
    print(random.choice(hot_songs))
                 
else:
    
    try:
        cluster = kmeans.predict(X_prep_less)
        narrowed_down = database_songs[(database_songs['clusters'] == cluster[0])]
        suggestion_list = narrowed_down['id'].tolist()
        selection = random.choice(suggestion_list)
        suggestion = database_songs[(database_songs['id']==selection)]
        output = suggestion[["name", "artist","url"]]
        print("May I suggest the following song:")
        for col_name in output.columns: 
            print(col_name+':', output[col_name]. value_counts(). idxmax())
        
    except:
        print("no recommendation")  

May I suggest the following song:
name: Batom De Ouro - Ao Vivo
artist: Os Barões Da Pisadinha
url: https://open.spotify.com/track/2KEZ1NCCy9utpQ8Dg1TTpk


In [97]:
# cluster = kmeans.predict(X_prep_less)
# cluster